<a href="https://colab.research.google.com/github/mcldwitt/workshops/blob/main/Animal_detection_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='#9E7BFF'> Animal detection


Run de volgende cel zodat de nodige libraries worden geinstalleerd

In [ ]:
!pip install ultralytics==8.1.34
!pip install gradio

In [ ]:
# Vervang door de code die je hebt gekopieerd uit je eigen Roboflow workspace

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="...")
project = rf.workspace("...").project("...")
version = project.version(1)

# dit lijntje laten staan!!!!
dataset = version.download("yolov8", "./dierenproject")

open `data.yaml` zoals in het document aangegeven staat


run de volgende cel  
waarden die je eventueel kan veranderen:  

*   `epochs` pas hiermee wel op, 1 epoch duurt ongeveer 1 minuut om het model te trainen, dus als deze op bv 20 staat dan zal je 20 minuten moeten wachten op het resultaat.
*   `batch` mag de waarde 2,4,6,8,16,32,64 of 128 zijn.



In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=10 imgsz=640 batch=64

Als je de volgende cel runt zou je een confusion matrix moeten zien. Indien dit niet zo is, contacteer een leerkracht.

In [ ]:
bfrom IPython.display import display, Image
Image(filename=f'/content/runs/detect/train/confusion_matrix.png', width=900)

In [ ]:
from ultralytics import YOLO
model = YOLO("runs/detect/train/weights/best.pt")

Laat ons het model eens uittesten in een kleine applicatie, run de volgende cel.

In [ ]:
import gradio as gr
import PIL.Image as Image
from ultralytics import ASSETS, YOLO

model = model
# model = YOLO("yolov8n.pt") goed werkend model

def predict_image(img, conf_threshold, iou_threshold):
    """Predicts and plots labeled objects in an image using YOLOv8 model with adjustable confidence and IOU thresholds."""
    results = model.predict(
        source=img,
        conf=conf_threshold,
        show_labels=True,
        show_conf=True,
        imgsz=640,
    )

    for r in results:
        im_array = r.plot()
        im = Image.fromarray(im_array[..., ::-1])

    return im


iface = gr.Interface(
    fn=predict_image,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Slider(minimum=0, maximum=1, value=0.25, label="Confidence threshold"),
    ],
    outputs=gr.Image(type="pil", label="Result"),
    title="Ultralytics Gradio",
    description="Upload images to check if there is an animal",
    theme = gr.themes.Glass(
    primary_hue="pink",
    secondary_hue="purple",
    neutral_hue="fuchsia",
),

)

if __name__ == "__main__":
    iface.launch(debug=True)

In [ ]:
import gradio as gr

gr.themes.builder()